In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os 
from datetime import timedelta
pd.set_option('display.max_columns', None)

data_path = str(Path(os.getcwd())) + "/data/"

def write_to_pickle(dataframe, name):
    dataframe.to_pickle(data_path + name + ".pickle")
def read_from_pickle(name): 
    return pd.read_pickle(data_path + name + ".pickle")

In [2]:
df = read_from_pickle('final_dataset')
df_no_censoring = read_from_pickle('final_dataset_no_censoring')

In [5]:
df.shape

(2635, 25)

In [6]:
df_no_censoring.shape

(2086, 25)

In [7]:
# Drop ID so that MICE imputation works
df.drop(['ID'], axis=1,inplace=True)
df_no_censoring.drop(['ID'], axis=1, inplace =True)

In [8]:
# checking how many observations we have with different numbers of NA data across variables
no_NAN = df.dropna(thresh = 35)
no_NAN_censoring = df.dropna(thresh = 35)
print(len(no_NAN), len(no_NAN_censoring))

df.drop("RX", axis = 1, inplace=True)
df.drop("CHEM", axis = 1, inplace=True)

df_no_censoring.drop("RX", axis = 1, inplace=True)
df_no_censoring.drop("CHEM", axis = 1, inplace=True)

0 0


In [9]:
#export dataset for imputation
df.to_csv('data/preimputation_dataset_with_censoring_18022018.csv', index=False)

In [9]:
df.to_csv('data/preimputation_datasets_without_censoring_18022018.csv',index=False)

here we will need to import the imputed dataset, which used life expectancy for the information it contains, and replace
this variable with the same column from the pre imputation dataset, since we are not interested in the imputed target variable values

In [ ]:
path_alex = ""
df_imputed = pd.read_csv(path_alex + "/imputed_dataset_with_censoring_18022018")
df_imputed['life_expectancy'] = df_final_kps['life_expectancy']
df_imputed['life_expectancy'].isnull().value_counts()

df_imputed.to_csv("Dropbox/Healthcare work Patric/imputed_dataset_with_censoring_16022018.csv")
df_imputed_knn = pd.read_csv("Dropbox/Healthcare work Patric/imputed_dataset_with_censoring_16.02.2018_kNN_before life expectancy reset.csv")
df_imputed_knn['life_expectancy'] = df_final_kps['life_expectancy']
df_imputed_knn['life_expectancy'].isnull().value_counts()
df_imputed_knn.to_csv("Dropbox/Healthcare work Patric/imputed_dataset_with_censoring_16022018_kNN.csv")

# Below is R imputation for MICE, Amelia, KNN

## MICE imputation

Set directory

setwd("Dropbox/Healthcare work Patric")

require mice package

require(mice)

load dataset

dataset = read.csv("imputation_set_24012018.csv", header=TRUE, na.strings = '')
summary(dataset)

load VIM library for missing data visualization

library(VIM)
aggr(dataset, prop = F, numbers = T)

matrixplot(dataset, interactive = F)

marginplot(dataset[,c("IK","IDH_TERT")])

code IDH_TERT and X1p19q_codel as factors so that R/MICE doesnt treat them as continuous

dataset$IDH_TERT <- as.factor(dataset$IDH_TERT)

dataset$X1p19q_codel <- as.factor(dataset$X1p19q_codel)

run mice imputation with 20 iterations

imp <- mice(dataset, maxit = 20)

inspect methods used for imputing each variable

imp$method

#Gender      Tumor_type     Tumor_grade        Gene_P53       Gene_Mgmt       Gene_Egfr       Gene_Mdm2 
#    ""              ""              ""       "polyreg"        "logreg"        "logreg"        "logreg" 
#     Gene_Cdk4        Gene_P16   Gene_Ihc_Atrx      Gene_Ch10Q       Gene_Ch9P  Tumor_Location  Tumor_Position 
#       "logreg"        "logreg"       "polyreg"        "logreg"       "polyreg"       "polyreg"       "polyreg" 
#   Surgery_type     Age_surgery life_expectancy             IDH            TERT        IDH_TERT    X1p19q_codel 
#      "polyreg"              ""              ""       "polyreg"        "logreg"       "polyreg"        "logreg" 
#             IK 
#          "pmm" 

Inspect convergenve

plot(imp)

stripplot(imp, pch = 20, cex = 1.2)

export the imputed dataset

write.csv(complete(imp), file = "imputed_dataset_no_censoring_16022018")

## Amelia imputation

setwd(Dropbox/Healthcare work Patric)

dataset = read.csv("imputation_dataset_no_censoring_24022018", header = TRUE, na.strings = '')

load Amelia

library(Amelia)

run imputation

noms = categorical variables

ords = ordinal variables

idvars = nominal variables that should not be imputed

if running Amelia on full dataset, it will throw an error. you need to remove either Tumor_type of Tumor_grade
since removing Tumor_type converges faster, I removed Tumor_grade by adding it to idvars option
Since both Tumor_type and Tumor_grade variables have no missing data, this does not hinder us at all

a.out <- amelia(dataset, m=1, noms = c("Gender", "Tumor_grade", "Gene_P53", "Gene_Mgmt", "Gene_Egfr", "Gene_Mdm2", 
                                       "Gene_Cdk4", "Gene_P16", "Gene_Ihc_Atrx", "Gene_Ch10Q", "Gene_Ch9P", 
                                       "Tumor_Location", "Tumor_Position", "Surgery_type", "IDH", "TERT", 
                                       "X1p19q_codel"), ords = c("IDH_TERT"), idvars = c("RX", "CHEM", "Tumor_type"))
                                       
                                       
get key messages post imputation, including "Normal EM convergence"

a.out    

get full summary of output dataset

summary(a.out)

Post imputation diagnostic graphs

plot(a.out, which.vars = 4:15)

overimpute(a.out, var = "IDH_TERT")

overimpute(a.out, var = "IK")

export dataset. Here a.out contains only one imputed dataset. If it contained 5, this would export each of them in a csv 
named file.stem+1, file.stem+2...

write.amelia(obj=a.out, file.stem = "imputed_dataset_no_censoring_24022018_Amelia")

we repeat the same with the censured dataset. However, this time Amelia throws a collinearity error again, even when run
with the same parameters as for non censored dataset
As such, we pass "life expectancy" as an idvars to avoid collinearity issue

a.out <- amelia(dataset, m=1, noms = c("Gender", "Tumor_grade", "Gene_P53", "Gene_Mgmt", "Gene_Egfr", "Gene_Mdm2", 
                                                                                "Gene_Cdk4", "Gene_P16", "Gene_Ihc_Atrx", "Gene_Ch10Q", "Gene_Ch9P", 
                                                                                "Tumor_Location", "Tumor_Position", "Surgery_type", "IDH", "TERT", 
                                                                                "X1p19q_codel"), 
                                                                                
                                                                                
                                                                                                   ords = c("IDH_TERT"), idvars = c("Tumor_type", "life_expectancy"))

## KNN Imputation

dataset <- read.csv("imputation_dataset_no_censoring_24022018", header=TRUE, na.strings = "")

dataset$IDH_TERT <- as.factor(dataset$IDH_TERT)

dataset$X1p19q_codel <- as.factor(dataset$X1p19q_codel)

imp <- kNN(dataset, k = 10)

write.csv(imp, "imputed_dataset_with_censoring_16.02.2018_kNN.csv")

Same but for censored dataset

dataset <- read.csv("imputation_dataset_with_censoring_24022018", header=TRUE, na.strings = "")

dataset$IDH_TERT <- as.factor(dataset$IDH_TERT)

dataset$X1p19q_codel <- as.factor(dataset$X1p19q_codel)

imp <- kNN(dataset, k = 10)

write.csv(imp, "imputed_dataset_with_censoring_26022018_kNN.csv")